# Teste Técnico para Estagio - Rafael Geraldini Francisco

#### Primeiro passo foi chamar algumas bibliotecas

In [1]:
import pandas as pd
import numpy as np
from collections import deque
import plotly.graph_objects as go

#### Irei definir uma classe Graph para tratar os dados fornecidos, irei assumir que são do tipo "undirect" e de mesmo pesos. Será representado por lista adjacente em um dicionario do python

In [2]:
class Graph:
    def __init__(self, q_vertex):                                    #Inicia a classe
        self.d_q_vertex = q_vertex                                   #Define a variavel interna
        self.d_vertex = range(self.d_q_vertex)                       #Tamanho

        self.d_ladj = {vertex: set() for vertex in self.d_vertex}    #Prepara o set

    def new_edge(self, vertex1, vertex2):                            #Comando para adicionar novo edge
        self.d_ladj[vertex1].add(str(vertex2))                       #Por ser undirect
        self.d_ladj[vertex2].add(str(vertex1))

#    def print_ladj(self):
#        for key in self.d_ladj.keys():
#            print("vertex", key, ": ", self.d_ladj[key])

#### Para calculo da "Closeness Centrality" irei utilizar o algoritmo A*, assim irei definir uma classe com ele onde tudo sera tratado

##### A função heuristica foi considerada igual a 1 para todos os nodes, visto que no calculo manual do caminho para um um grupo de 10 edges encontrou uma distancia media de 2, assim não aparece no calculo

In [3]:
class Astar:
    def __init__(self, ladj):
        self.ladj = ladj

    def l_edges(self, v):
        return self.ladj[v]

    def alg_astar(self, source, target):
        vistos_nexaminados = set([source])      #Visitado mas ainda falta nodes
        vistos_examinados = set([])             #Visitado completamente
        d = {}                                  #Distancia dos nodes
        d[source] = 0                           #Distancia source source
        prox = {}                               #Na proximidade
        prox[source] = source

        while len(vistos_nexaminados) > 0:
            n = None

            for v in vistos_nexaminados:        #Encontra a menor distancia
                if n == None or d[v] < d[n] :
                    n = v;

            if n == target:                     #Se chegou, define o caminho usado
                caminho = []

                while prox[n] != n:
                    caminho.append(n)
                    n = prox[n]

                caminho.append(source)

                caminho.reverse()

                return caminho

            for m in self.l_edges(n):           #Para a proximidade
                if m not in vistos_nexaminados and m not in vistos_examinados:
                    vistos_nexaminados.add(m)
                    prox[m] = n
                    d[m] = d[n] + 1
                else:
                    if d[m] > d[n] + 1:
                        d[m] = d[n] + 1
                        prox[m] = n

                        if m in vistos_examinados:
                            vistos_examinados.remove(m)
                            vistos_nexaminados.add(m)

            
            vistos_nexaminados.remove(n)        #Quando terminou com o vertice
            vistos_examinados.add(n)

        return None

#### Importei o arquivo fornecido como um Dataframe do pandas

In [4]:
df = pd.read_csv('edges', sep=' ',header=None) #Importa o arquivo que deve estar na mesma pasta
edges = len(df)  #Encontra a quantidade de edges
dfagr = df.groupby(0).agg(list).to_dict().get(1) #agrupa o array por nodes
vertices = len(dfagr) #Encontra a quantidade de vertices

#### Inicia o Graph

In [5]:
G = Graph(vertices)

npa=np.array(df)   #Transforma o dataframe em um array do numpy
edg= range(edges)
for n in edg:
    G.new_edge(npa[n,0],npa[n,1])

keys_values = G.d_ladj.items()
Cms = {str(key): value for key, value in keys_values} #Transforma a chave do dicionario em string para uso no A*

#### Inicia o Algoritmo A*

In [6]:
GAstar = Astar(Cms)    #cria a entidade da classe Astar                                         
vtc = range(vertices)  #variavel com a quantidade de vertices
CC = []   #inicia a lista onde será salvo CC
for x in vtc:      #para cada vertice
    dist = 0       #reinicia a variavel
    for y in vtc:  #para cada vertice
        if x != y: #distancia do vertice pra ele mesmo não entra
            dist = dist + len(GAstar.alg_astar(str(x), str(y))) #dist é somatorio da distancia entre pelo algoritmo
    CC.append((x, 1/dist, (vertices-1)/dist))   #adiciona o vertice atual, cc e cc normalizada na lista
    
    

#### Organiza a lista e exporta os dados

In [7]:
CC_sorted = sorted(CC, key=lambda CC: CC[1], reverse=True) #ordena pelo maior Closeness centrality

In [8]:
dff = pd.DataFrame(CC_sorted)  #transforma a lista em um dataframe do pandas
names = ['Vértice', 'CC', 'CC Normalizada']  #nomes das colunas
dff.columns = names  #nomeia as colunas
dff.to_csv('CC_sorted.csv', index=False) #cria um csv com os dados para uso em outros software

#### Network Graph

In [9]:
fig = go.Figure() #Inicia a figura
vn=0
for op in range(vertices):  # interage para todos os vértices, e usa o dicionario para encontrar as ligações
    vat = int(dff.iat[op,0])
    vzn = dfagr[vat]
    xi = (op % 10) + 1
    yi = (op // 10) +1
    for opr in range(len(vzn)):
        tg = vzn[opr]
        xt = (tg % 10) + 1
        yt = (tg // 10) +1
        x=[xi, xt]
        y=[yi, yt]
        fig.add_trace(        #Adiciona o traço como representação de edges
            go.Scatter(
                mode='lines',
                x=x,
                y=y,
                line=dict(
                    color='black',
                    width=1,
                ),
                showlegend=False
            )
        )


In [10]:
x=[] #inicia vetores para x, y e nomes
y=[]
nome=[]
for op in range(1,11):   #gera a posição e o nome de cada node
    for opr in range(1,11):
        y.append(op)
        x.append(opr)
        nome.append(str(dff.iat[int((op+opr)-2),0]))

fig.add_trace(   #adiciona cada node na figura com os edges
            go.Scatter(
                mode='markers+text',
                text=nome,
                textposition="middle center",
                textfont = dict(
                    color='black',
                    size=14
                ),
                x=x,
                y=y,
                marker=dict(
                    color='green',
                    size=40,
                    line=dict(
                        color='red',
                        width=1
                    )
                ),
                showlegend=False
            )
        )
fig.update_layout(   #Ajusta o tamanho da figura
    autosize=False,
    width=800,
    height=800,
)

fig.write_html("Graph.html") #Exporta como interativo e estatico
fig.write_image("Graph.png")